## CUSTOMER SEGMENTATION

Firstly, declaring necessary libraries

In [1]:
# for data analysis
import pandas as pd
import numpy as np

import json     # for reading the key inside the json formatted file

# for data visualization
import matplotlib.pyplot as plt

import pyodbc   # for connecting database

> Connecting to Database

Pyodbc library handles the connection between Jupyter notebook and MS SQL Server. SQL Server's key is hidden inside the json file.

In [2]:
f = open('log.json')
sql_key = json.load(f)     # returns JSON object as a dictionary

cnxn = pyodbc.connect(sql_key['key'])     # establish a connection
crsr = cnxn.cursor()                      # cursor enables to send command

> Sending Queries 

Queries are sent with respect to the decisions given on analytics phase. G and T type company data are retrieved separately.

For G (Şahıs)

In [35]:
gk_query= """SELECT MUSTERI_ID, ID, CEK_NO, CEK_TUTAR, VADE_GUN, BK_LIMIT, BK_RISK,
            BK_GECIKMEHESAP, BK_GECIKMEBAKIYE
            FROM dbo.dataset
            WHERE SIRKET_TURU LIKE 'G' """

g_company_type_df = pd.read_sql(gk_query, cnxn)

C:\Users\emirh\AppData\Local\Temp\ipykernel_2776\1577447570.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  g_company_type_df = pd.read_sql(gk_query, cnxn)


In [17]:
g_company_type_df.head(10)

,MUSTERI_ID,ID,CEK_NO,CEK_TUTAR,VADE_GUN,BK_LIMIT,BK_RISK,BK_GECIKMEHESAP,BK_GECIKMEBAKIYE
0,11800527,2457932,70331933,20000,100,0,0,0,0
1,12024009,2457933,3014103,25000,126,2213,226,2,1012
2,11800527,2457934,7031933,20000,100,0,0,0,0
3,11724283,2457936,7198012,23000,151,52100,9481,3,3575
4,11879266,2457937,9090937,10000,151,280101,211301,3,4128
5,11879266,2457938,9090936,13458,121,280101,211301,3,4128
6,11854083,2457939,4535918,5200,4,0,0,0,0
7,11654711,2457942,88624,110000,181,215261,150619,1,847
8,11723432,2457943,3331313,5300,121,104384,65599,3,2591
9,11577211,2457944,4602258,11000,187,215860,85589,4,6379


For T (Tüzel)

In [31]:
tk_query= """SELECT MUSTERI_ID, ID, CEK_NO, CEK_TUTAR, VADE_GUN, TK_NAKDILIMIT, TK_NAKDIRISK, TK_GAYRINAKDILIMIT, TK_GAYRINAKDIRISK, TK_GECIKMEHESAP, TK_GECIKMEBAKIYE FROM dbo.dataset WHERE SIRKET_TURU LIKE 'T' """
t_company_type_df = pd.read_sql(tk_query, cnxn)

C:\Users\emirh\AppData\Local\Temp\ipykernel_18068\595317809.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  t_company_type_df = pd.read_sql(tk_query, cnxn)


### MACHINE LEARNING

The ML is going to be implemented to segment customer portfolio into clusters based on their risks. Firstly, the customer portfolio divided into two groups: T type and G type customers. Due to the differences between their attributes, this was inevitable step to be done. Also, it is crucial to define type of machine learning. Due to the attributes, it will be unsupervised learning. As observed, all attributes will be used are features. For providing accurate solution, we obtained that classification of the customer portfolio is a must requirements. In this sense, **K-means** is going to implemented.

Next, the datasets will be prepared for the clustering. There is a need of deriving new attributes by using existing ones. Thus, the feature extraction must be done before putting data into model. In addition to this, data scaling is another significant task to complete. According to Dong, Zhang, and Chen (2020), *data scaling is a technique used in machine learning to adjust the range of features in a dataset, typically to a common scale between zero and one or -1 and 1.* The purpose of data scaling is to standardize the data, mitigate the impact of outliers, and ensure that all features are treated equally during model training. As stated in the paper, we are looking forward to handle large range of magnitudes with this method.

Also, data will be divided into two as training and test for ML. Sarkar and Bali (2020) stated that training dataset is used to teach the algorithm to recognize patterns in the data, while the testing dataset is used to evaluate the algorithm's ability to generalize to new data. We are going to determine whether the model is good fit or not by using test dataset. It is a common fact that overfitting or underfitting models my occur during ML researches.

Moreover, the optimal number of clusters are going to be obtained by using Elbow Method from the data. The elbow method is a critical technique for selecting the optimal number of clusters in k-means clustering, as it provides an objective way to determine the appropriate number of clusters and helps to prevent overfitting and improve the interpretability of the resulting clusters (Tang & Zhang, 2019). After this, we are going to train the model with the train dataset.

After these steps, the accuracy of K-means model must be found out. Also, other appropriate unsupervised ML modelling techniques are going to be compared. The most fitting model's results are going to be saved into the database. The database is going to be integrated our data-oriented web application for strong Business Intelligence presentation.

To conclude, our steps are;

*   Feature Extraction
*   Scaling Data
*   Training/Test Set Division
*   Elbow Method
*   Accuracy Calculation
*   Comparing K-means with Other Models
*   Saving Results into Database
*   Presenting Results via Streamlit Web-App

> Handle Data Types

Firstly, we have to assign data types before ML operations

In [36]:
g_company_type_df.info()    # to see the data types and missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229538 entries, 0 to 229537
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   MUSTERI_ID        229538 non-null  object
 1   ID                229538 non-null  int64 
 2   CEK_NO            229538 non-null  object
 3   CEK_TUTAR         229538 non-null  object
 4   VADE_GUN          229538 non-null  object
 5   BK_LIMIT          229538 non-null  object
 6   BK_RISK           229538 non-null  object
 7   BK_GECIKMEHESAP   229538 non-null  object
 8   BK_GECIKMEBAKIYE  229538 non-null  object
dtypes: int64(1), object(8)
memory usage: 15.8+ MB


In [37]:
# there are several characters in the MUSTERI_ID column, so we need to remove them
g_company_type_df['MUSTERI_ID']=g_company_type_df['MUSTERI_ID'].str.replace(r'\D', '').astype(int)
g_company_type_df['ID']=g_company_type_df['ID'].astype(int)

# VADE_GUN represents the amount of day, so it is an integer
g_company_type_df['VADE_GUN']=g_company_type_df['VADE_GUN'].astype(int)
g_company_type_df['CEK_TUTAR']=g_company_type_df['CEK_TUTAR'].replace(',','.', regex=True).astype(float)

# converting into float to use on ML model later
g_company_type_df['BK_LIMIT']=g_company_type_df['BK_LIMIT'].astype(float)
g_company_type_df['BK_RISK']=g_company_type_df['BK_RISK'].astype(float)
g_company_type_df['BK_GECIKMEHESAP']=g_company_type_df['BK_GECIKMEHESAP'].astype(float)
g_company_type_df['BK_GECIKMEBAKIYE']=g_company_type_df['BK_GECIKMEBAKIYE'].astype(float)

C:\Users\emirh\AppData\Local\Temp\ipykernel_2776\1768993693.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  g_company_type_df['MUSTERI_ID']=g_company_type_df['MUSTERI_ID'].str.replace(r'\D', '').astype(int)


*CEK_NO* has several improper values, so we need to handle errors and convert it into integer

In [ ]:
# used these queries to spot the non-integer values
"""SELECT [CEK_NO]
FROM [faktoring_db].[dbo].[dataset]
WHERE CEK_NO like '%[^a-Z0-9]%'

SELECT CEK_NO, COUNT(*) as count
FROM [faktoring_db].[dbo].[dataset]
WHERE CEK_NO like '%[^a-Z0-9]%'
GROUP BY CEK_NO
HAVING COUNT(*) > 0

//OR USE THIS, MORE EFFICIENT RESULTS

SELECT [ID],[CEK_NO]
FROM [faktoring_db].[dbo].[dataset]
WHERE ISNUMERIC([CEK_NO]) <> 1;"""

Solved the issue on MS SQL with the following script below:

In [ ]:
#...and solved the issue by using the following queries
"""
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 1
WHERE [ID] = 2770334;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 2
WHERE [ID] = 2494076;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 3
WHERE [ID] = 2501347;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 4
WHERE [ID] = 2551433;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 5
WHERE [ID] = 2551434;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 6
WHERE [ID] = 2633345;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 7
WHERE [ID] = 2689531;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 8
WHERE [ID] = 2699745;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 9
WHERE [ID] = 2703751;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 10
WHERE [ID] = 2715934;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 11
WHERE [ID] = 2828009;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 12
WHERE [ID] = 2828010;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 13
WHERE [ID] = 2828011;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 14
WHERE [ID] = 2828012;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 15
WHERE [ID] = 2828013;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 16
WHERE [ID] = 2516070 OR
	[ID] = 2525905 OR
	[ID] = 2573048 OR
	[ID] = 2685207 OR
	[ID] = 2727249;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 17
WHERE [ID] = 2741161;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 18
WHERE [ID] = 2766289;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 19
WHERE [ID] = 2809942;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 20
WHERE [ID] = 2815551;

UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 21
WHERE [ID] = 2850473;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 22
WHERE [ID] = 2911050;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 23
WHERE [ID] = 2912888;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 24
WHERE [ID] = 2912889;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 25
WHERE [ID] = 2922915;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 26
WHERE [ID] = 2922916;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 27
WHERE [ID] = 2923795;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 28
WHERE [ID] = 2703536;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 29
WHERE [ID] = 2740283;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 30
WHERE [ID] = 2622979;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 31
WHERE [ID] = 2740284;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 32
WHERE [ID] = 2740285;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 33
WHERE [ID] = 2740286;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 34
WHERE [ID] = 2740287;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 35
WHERE [ID] = 2740288;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 36
WHERE [ID] = 2740289;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 37
WHERE [ID] = 2740290;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 38
WHERE [ID] = 2780745;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 39
WHERE [ID] = 2782835;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 40
WHERE [ID] = 2782836;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 41
WHERE [ID] = 2823409;
UPDATE [faktoring_db].[dbo].[dataset]
SET [CEK_NO] = 42
WHERE [ID] = 2823497;
"""

In [39]:
# finally, the script is executed successfully
g_company_type_df['CEK_NO']=g_company_type_df['CEK_NO'].astype('int64')

In [41]:
# checking uniqueness of ID
g_company_type_df['ID'].is_unique   # check if there is any duplicate in ID

True

In [40]:
g_company_type_df.info()    # to see the data types after the changes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229538 entries, 0 to 229537
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   MUSTERI_ID        229538 non-null  int32  
 1   ID                229538 non-null  int32  
 2   CEK_NO            229538 non-null  int64  
 3   CEK_TUTAR         229538 non-null  float64
 4   VADE_GUN          229538 non-null  int32  
 5   BK_LIMIT          229538 non-null  float64
 6   BK_RISK           229538 non-null  float64
 7   BK_GECIKMEHESAP   229538 non-null  float64
 8   BK_GECIKMEBAKIYE  229538 non-null  float64
dtypes: float64(5), int32(3), int64(1)
memory usage: 13.1 MB


> Feature Extraction

We must create a new attribute called **BK_ORAN** which is *derived by diving BK_RISK to BK_LIMIT*

In [ ]:
# use this when BK_LIMIT is not 0:
# g_company_type_df['BK_ORAN']=g_company_type_df['BK_RISK']/g_company_type_df['BK_LIMIT']
# else handle division error

> Scaling Data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
def dataframe_scaler(df):
    scaler.fit(df)      # df is not exist yet, volatile to change later
    return scaler.transform(df)

> Elbow Method

We aim to maximize the efficiency of segmentation while minimizing the number of clusters. In this sense, Elbow Method is crucial concept to satisfy this requirement.

In [11]:
# importing scikit-learning library for the K-means model
from sklearn.cluster import KMeans

In [ ]:
def find_optimal_clusters(df, maximum_K):
    clusters_centers = []   # appending inertia value coming from the model
    k_values = []           # putting k values from 0 to maximum K

    for k in range(1, maximum_K):
        kmeans_model = KMeans(n_clusters = k)
        kmeans_model.fit(df)

        clusters_centers.append(kmeans_model.inertia_)
        k_values.append(k)

    return clusters_centers, k_values

Note About **Inertia**:

Inertia measures how well a dataset was clustered by K-Means. It is calculated by measuring the distance between each data point and its centroid, squaring this distance, and summing these squares across one cluster. A good model is one with low inertia AND a low number of clusters (K) *-Codeacademy*

In the next step, illustrating an elbow method for the data

In [ ]:
def generate_elbow_plot(clusters_centers, k_values):
    figure = plt.subplots(figsize = (12, 6))
    plt.plot(k_values, clusters_centers, 'o-', color = 'blue')
    plt.xlabel("Number of Clusters")
    plt.ylabel("Cluster Inertia")
    plt.title("Elbow Plot of Model")
    plt.show()

In [ ]:
clusters_centers, k_values = find_optimal_clusters(df, 16)
generate_elbow_plot(clusters_centers, k_values)